# Combining Datasets Tutorial

In this tutorial, we are going to see how to use BastionLab's vstack function to join RemoteLazyFrames vertically. This can be particularly useful if you want to query or train models on a combination of datasets from different parties. The only requirement is that datasets must have the same column names/types. You can learn about dataset preparation including changing column types, names and adding and removing columns in out data cleaning tutorial.

## Pre-requisites

### Technical requirements

- You have installed BastionLab Server and Client.

If not, you can go check out our [installation page](https://bastionlab.readthedocs.io/en/latest/docs/getting-started/installation/) or run the code block below: 

In [1]:
!pip install bastionlab
!docker run -it -p 50056:50056 -d mithrilsecuritysas/bastionlab:latest

In [ ]:
# To run the server
from bastionlab import Connection

connection = Connection("localhost")
client = connection.client

### Upload the dataframe to the server

For this tutorial, we'll create three simple Polars dataframes with "Element" and "Melting Point (K)" columns.

In [4]:
import polars as pl

df1 = pl.DataFrame(
    {
        "Element": ["Copper", "Silver", "Gold"],
        "Melting Point (K)": [1357.77, 1234.93, 1337.33],
    }
)

df2 = pl.DataFrame(
    {"Element": ["Platinum", "Palladium"], "Melting Point (K)": [2041.4, 1828.05]}
)

df3 = pl.DataFrame({"Element": ["Titanium"], "Melting Point (K)": [1945.0]})

Now we need to send these Polar dataframes to the server and get back our RemoteLazyFrame instances which we will be working with in the rest of the tutorial.

Since this is not the focus of this tutorial, we'll quickly set up an open safety policy that allows us to demonstrate features without having to approve any data access requests. We want to underline that **this policy is not suitable for production.**

In [5]:
from bastionlab.polars.policy import Policy, TrueRule, Log

policy = Policy(safe_zone=TrueRule(), unsafe_handling=Log())

rdf1 = client.polars.send_df(df1, policy=policy)
rdf2 = client.polars.send_df(df2, policy=policy)
rdf3 = client.polars.send_df(df3, policy=policy)

So now we will test out vstack by adding the second dataframe, with the Platinum and Palladium elements, to our first one, with Copper, Silver and Gold. We simply change rdf1 to equal the RemoteLazyFrame returned by our vstack function.

As you can see when we .collect().fetch() our RemoteLazyFrame, the second dataframe has now been appended to the bottom of the first one.

In [6]:
rdf1 = rdf1.vstack(rdf2)
rdf1.collect().fetch()

Element,Melting Point (K)
str,f64
"""Copper""",1357.77
"""Silver""",1234.93
"""Gold""",1337.33
"""Platinum""",2041.4
"""Palladium""",1828.05


Next we will add our third RemoteLazyFrame containing Titanium to rdf1, two times!

As you can see, we now have all the previous elements, plus two lots of Titanium at the end.

In [7]:
rdf1 = rdf1.vstack(rdf3)
rdf1 = rdf1.vstack(rdf3)
rdf1.collect().fetch()

Element,Melting Point (K)
str,f64
"""Copper""",1357.77
"""Silver""",1234.93
"""Gold""",1337.33
"""Platinum""",2041.4
"""Palladium""",1828.05
"""Titanium""",1945.0
"""Titanium""",1945.0


And that's it for this tutorial. Finally, we will now simply close the connection.

In [8]:
connection.close()